In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F 
import mmap
import random
import pickle 
import argparse
parser = argparse.ArgumentParser(description = 'This is a deomnstration Program')
parser.add_argument('-bs',type=str, required = True, help = 'Batch Size' )
args = parser.parse_args()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'batch size:{args.batch_size}')
print(device)
block_size= 64 
batch_size = args.batch_size
learning_rate = 3e-4
max_iters = 200
eval_iters = 250
dropout = 0.2
n_embd = 384
n_layer = 8
n_head = 8
dropout  = 0.2

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))
usage: ipykernel_launcher.py [-h] -bs BS
ipykernel_launcher.py: error: the following arguments are required: -bs


SystemExit: 2

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [75]:
chars = ""
with open('openwebtext/vocab.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    chars = sorted(set(text))

print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
80


In [36]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])



tensor([28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,  0,
         1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,  1,
         1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50,
        25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38,
        28,  1, 39, 30,  1, 39, 50,  9,  1, 39])


In [77]:

def get_random_chunk(split):
    filename = "openwebtext/train_split.txt" if split == 'train' else 'openwebtext/val_split.txt'

    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access= mmap.ACCESS_READ) as mm:

            file_size = len(mm)
            start_pos = random.randint(0,(file_size) - block_size*batch_size)

            mm.seek(start_pos)
            block = mm.read(block_size* batch_size-1  )
            decoded_block = block.decode('utf-8', error= 'ignore').replace('\r', '')
            data = torch.tensor(encode(decoded_block), dtype = torch.long)
    return data 


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device) , y.to(device)
    return x,y
    
x,y = get_batch('train')

print('inputs:')
print(x)
print('targrets:')
print(y)
    

inputs:
tensor([[72,  1, 62,  ..., 61, 62, 60],
        [ 1, 58, 67,  ...,  1, 66, 62],
        [67,  1, 54,  ..., 73,  1, 66],
        ...,
        [75, 62, 73,  ..., 67, 60, 72],
        [73,  1, 66,  ..., 58,  1, 61],
        [59,  0, 69,  ..., 60, 54, 62]])
targrets:
tensor([[ 1, 62, 67,  ..., 62, 60, 61],
        [58, 67, 57,  ..., 66, 62, 57],
        [ 1, 54, 67,  ...,  1, 66, 74],
        ...,
        [62, 73, 54,  ..., 60, 72,  1],
        [ 1, 66, 58,  ...,  1, 61, 58],
        [ 0, 69, 71,  ..., 54, 62, 67]])


In [78]:
class FeedForward(nn.Module):
     def __init__(self, n_embd):
          super().__init__()
          self.net = nn.Sequential(
               nn.Linear(n_embd, 4 * n_embd),
               nn.ReLU(),
               nn.Linear(4 * n_embd, n_embd),
               nn.Dropout(dropout),
          )

     def forward(self, x):
          return self.net(x)
          

class Head (nn.Module):
     def __init__(self,  head_size ):
          super().__init__()
          self.key = nn.Linear(n_embd, head_size, bias = False)
          self.query = nn.Linear(n_embd, head_size, bias = False)
          self.value = nn.Linear(n_embd, head_size, bias =False)
          self.register_buffer('trl', torch.tril(torch.ones(block_size, block_size)))

          self.dropout = nn.Dropout(dropout)

     def forward(self, x):
          B,T,C = x.shape
          k = self.key(x)
          q = self.query(x)
          wei = q @ k.transpose(-2,-1) * k.shape[-1] ** -0.5
          wei = wei.masked_fill(self.trl[:T, :T] == 0, float('-inf'))
          wei = F.softmax(wei, dim = -1)

          v = self.value(x)
          out = wei @ v

          return out
     

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads,head_size): 
          super().__init__()
          self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
          self.proj = nn.Linear(head_size * num_heads, n_embd, bias = False)
          self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.concat([head(x) for head in self.heads], dim =-1) #(B,T,F )---> ([h1, h1, h1, h2, h2, h2, h3, h3, h3, h4, h4, h4])
        out = self.dropout(self.proj(out))
        return out
          


class Block(nn.Module):
     def __init__(self, n_embd, n_head):
          super().__init__()
          head_size = n_embd // n_head
          self.sa = MultiHeadAttention(n_head, head_size)
          
          self.ffwd  = FeedForward(n_embd)
          self.ln1 = nn.LayerNorm(n_embd)
          self.ln2 = nn.LayerNorm(n_embd)

     def forward(self, x):
          y = self.sa(x)
          x = self.ln1(x+y)
          y = self.ffwd(x)
          x = self.ln2(x+y)
          
          return x


class GPTLanguageModel(nn.Module):
       def __init__(self, vocab_size):
           super().__init__()
           self.token_embeddings = nn.Embedding(vocab_size, n_embd)
           self.position_embeddings_table = nn.Embedding(block_size, n_embd)
           self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

           self.ln_f = nn.LayerNorm(n_embd)
           self.lm_head = nn.Linear(n_embd, vocab_size)
           self.apply(self._init_weights)

       def _init_weights(self, module):
           if isinstance(module, nn.Linear):
               torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
               if module.bias is not None:
                   torch.nn.init.zeros_(module.bias)
           elif isinstance(module, nn.Embedding):
                    torch.nn.init.normal_(module.weight, mean = 0.0, std =  0.02)
                   
       def forward(self, index, targets=None):
           B, T = index.shape
           token_emb = self.token_embeddings(index)
           pos_emb = self.position_embeddings_table(torch.arange(T, device=device))
           x = token_emb + pos_emb
           x = self.blocks(x)
           x = self.ln_f(x)
           logits = self.lm_head(x)


           # logits has shape (B, T, C) where C == vocab_size
           if targets is None:
               return logits, None

           B, T, C = logits.shape
           logits = logits.view(B * T, C)
           targets = targets.view(B * T)
           loss = F.cross_entropy(logits, targets)
           return logits, loss      

       def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            # Crop the context to block_size if it is longer.
            index_cond = index if index.shape[1] <= block_size else index[:, -block_size:]
            logits, _ = self(index_cond)
            # Focus on predictions for the last time step
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=-1)
        return index



         

In [79]:
    
model = GPTLanguageModel(vocab_size)
m = model.to(device)
print('Loading Learnable Parameters')
with open('model-01.pkl', 'rb') as f:
    pickle.load(f)
print('Model Loaded')
m= model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


743]oE3Od!&
W2C;rL*;E4vuw[q4w5Cfaf
]o1dFOCiPetAh[zcPP6*M.N4NW8m:jj LlzXhQRX4MOPIQEG(_CxCtLb(EO;W(fp-b:GxgQY-.Zo_3cy(7X!YQA,T.;LozlNHlp4'40"Wo6*Cu25!qg-dBO)o1taiZkHF60]rB-
CSYr6jcEY9i4qYD:,c?D(qWPP]';6*K3MMcELo6PPdZI?KH?*0-mrpRhq2Gih&iDrjV-B!8kDKSMlWOX0lo.:xav(rF(ojT0Dm0r?4Wv" D5HYjNr6j-g,FoZJw84i!5s
cYi96Zqt!.6z?VZKn XjhIR2ve-1S1RihC xnj0KECW4HGih7Ua;b)QOBj;kZD.ie:WI5-IpD?BiD,*DS)(;0N"qQC]8Gupqskxt6ProrU?Hw92.?isqJ6qfCQ!8EobHt:HNpid;&2m(VMnO0ykl_XKUqvVwtHja0iC9iUMLpq 1p?SkmmphwHd (r ]]?*n(R4.Q";


In [71]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

    

In [80]:

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
with open('model-01.pkl', 'wb') as f:
    pickle.dump(model,f)
print('model saved')

step 0: train loss 4.4483, val loss 4.4470
step 250: train loss 1.6727, val loss 1.8305
step 500: train loss 1.3159, val loss 1.6016
step 750: train loss 1.1195, val loss 1.5587
step 1000: train loss 0.9719, val loss 1.6050
step 1250: train loss 0.8151, val loss 1.6972
step 1500: train loss 0.6644, val loss 1.8327
step 1750: train loss 0.5399, val loss 1.9876
step 2000: train loss 0.4289, val loss 2.1893
step 2250: train loss 0.3534, val loss 2.3530
step 2500: train loss 0.3028, val loss 2.5364
step 2750: train loss 0.2756, val loss 2.6397
0.3649386763572693


In [81]:
context = torch.zeros((1,1) , dtype= torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 5000)[0].tolist())


print(generated_chars)




wings, which they had enemied, and he pretty graw them up in jail over, and the five bird, which they
poweced over the floor a in the floor.

"Where?" asked the man's voice of the strange wizard, that I don't think children work," remarked Zeb,
who had been until now been fore, the piggeeting of the raial of lithe unnear, and when they
made neares and hurt being for the air and began to make the best of
it.

"All were!" plates the wooden with attent once gent I am
incline."

"But you must remember you had the Country?" he enquired.

"No, my granerly, and watered in hand, and her adventure, we performing strap and
the buggy with snapped.

"Did you not wear green white fows, perhaped being one cut seems to rest, for the climb was rapider fastened
with was covered that pedigreened very conquered.

"Now, Billina!" she answered; "but that is, if there is anxiousnted to have
the boy eating his watch; and Zeb carried her with kind of the cold, the way
and not to with the star one, noting the

In [33]:
x = torch.tensor([10], dtype = torch.float32)
y = F.tanh(x)

print(y)


tensor([1.])
